In [ ]:
!pip install -q 'git+https://github.com/davies-w/chatspot.git'

In [ ]:
import chatspot

SPOTIFY_CLIENT_ID = ""
SPOTIFY_CLIENT_SECRET = ""
OPENAPI_API_KEY = ""

spotify_client = chatspot.login(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, OPENAPI_API_KEY)

vibe = "40s something disco"
validated_songs, target1, data1, songstr1, recommended_songs, target3, data3, songstr3 = chatspot.get_recommendations_by_vibe(spotify_client, vibe)

print(f"{vibe}:")
for s in songstr1:
  print(s)
print("")

vibe = "depressed teenager angst"
vs, target2, data2, songstr2, rs, target4, data4, songstr4 = chatspot.get_recommendations_by_vibe(spotify_client, vibe)
print(f"{vibe}:")
for s in songstr2:
  print(s)
print("")

In [ ]:
#@title TSNE

#https://www.datatechnotes.com/2020/11/tsne-visualization-example-in-python.html
#https://distill.pub/2016/misread-tsne/

from sklearn.manifold import TSNE
from keras.datasets import mnist
from sklearn.datasets import load_iris
from numpy import reshape
import seaborn as sns
import pandas as pd  
import numpy as np
import plotly.express as px

#PLAY WITH THIS!
x=np.array(data1 +  data2 + data3 + data4)
y=np.array(target1 + target2  + target3 + target4)
songnames=np.array(songstr1 + songstr2 + songstr3+ songstr4)

perplexity=4 # expected number of classes basically.
tsne = TSNE(n_components=2, n_iter=5000, verbose=0, random_state=123, perplexity=perplexity)
z = tsne.fit_transform(x) 

df = pd.DataFrame(x, columns = chatspot.get_features())
df["vibe"] = y
df["songnames"] = songnames
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

df['songnames']
#
#
title='Spotify Feature T-SNE projection'
hover_data={'vibe': False, 'comp-1':False, 'comp-2':False}
for f in chatspot.get_features():
  hover_data[f]=True

fig = px.scatter(df, hover_name='songnames', hover_data=hover_data, title=title,
                 x="comp-1", y="comp-2", color='vibe', width=1000, height=800,
                 color_discrete_sequence=["red", "green", "salmon", "lightgreen"])
#fig.update_traces(hovertemplate='%{hovertext}<extra></extra>',
#                  marker=dict(size=10))
fig.update_layout(title={'font_family': "Arial",
                         'x':0.45,
                         'y': 0.92,
                        'xanchor': 'center' })
fig.update_yaxes(tickvals=[], showgrid=False, visible=False)
fig.update_xaxes(tickvals=[], showgrid=False, visible=False)

fig.show()

In [ ]:
#@title PCA

#https://mclguide.readthedocs.io/en/latest/sklearn/clusterdim.html

import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA


x=np.array(data1 +  data2 + data3 + data4)
df = pd.DataFrame(x, columns = chatspot.get_features())

# Normalize Features to -1, 1, then Normalize Members to unit length, the
# Dimesionality reduction to 2
T0 = preprocessing.RobustScaler().fit_transform(df)
T1 = preprocessing.Normalizer().fit_transform(T0)
pca_model = PCA(n_components=2)
T2 = pca_model.fit_transform(T1) # fit the model


# store the values of PCA component in variable: for easy writing
xvector = pca_model.components_[0] * max(T2[:,0])
yvector = pca_model.components_[1] * max(T2[:,1])
columns = df.columns

df["vibe"] = y
df["songnames"] = songnames
df["comp-1"] = T2[:, 0]
df["comp-2"] = T2[:, 1]
df['songnames']

#
#
title='Spotify Feature PCA projection'
hover_data={'vibe': False, 'comp-1':False, 'comp-2':False}
for f in chatspot.get_features():
  hover_data[f]=True

fig = px.scatter(df, hover_name='songnames', hover_data=hover_data, title=title,
                 x="comp-1", y="comp-2", color='vibe', width=1000, height=800,
                 color_discrete_sequence=["red", "green", "salmon", "lightgreen"])
#print("plotly express hovertemplate:", fig.data[0].hovertemplate)
#fig.update_traces(hovertemplate='%{hovertext}%{danceability}<extra></extra>',
#                  marker=dict(size=10))
fig.update_layout(title={'font_family': "Arial",
                         'x':0.45,
                         'y': 0.92,
                        'xanchor': 'center' })

fig.update_yaxes(tickvals=[], showgrid=False, visible=False)
fig.update_xaxes(tickvals=[], showgrid=False, visible=False)

dim_sizes = []
for i in range(len(columns)):
  dim_sizes.append(math.sqrt(pow(xvector[i],2) + pow(yvector[i],2)))
dim_sizes = sorted(dim_sizes, reverse=True)
third = dim_sizes[3]

arrowsize = 1
arrowhead = 1
arrowscale = 1
for i, feature in enumerate(columns):
        if math.sqrt(pow(xvector[i],2) + pow(yvector[i],2)) < third:
          continue
        fig.add_annotation(
            ax=0, ay=0,
            axref="x", ayref="y",
            x=xvector[i]*arrowscale,
            y=yvector[i]*arrowscale,
            showarrow=True,
            arrowsize=arrowsize,
            arrowhead=arrowhead,
            xanchor="right",
            yanchor="top"
        )
        fig.add_annotation(
            x=xvector[i]*arrowscale,
            y=yvector[i]*arrowscale,
            ax=0, ay=0,
            xanchor="center",
            yanchor="bottom",
            text=feature,
            yshift=5,
        )



fig.show()